In [96]:
import re
import sys
import pprint
import json
from IPython.display import display
from colorama import Fore, Back, Style

In [197]:
with open('./results/lab1.json') as lab1_file:
    data = json.load(lab1_file)
tokens_chain = data['chain']
service_words = data['tables']['service_words']
operations = data['tables']['operations']
separators = data['tables']['separators']
constants = data['tables']['constants']
identifiers = data['tables']['identifiers']

nxtsymb = []
token_index = -1
wrapper()

['W', 0, 'program']
program
['I', 0, 'trivial']
['R', 4, ';']
['W', 1, 'var']
['I', 1, 'a']
var
['R', 3, ':']
['W', 3, 'integer']
var_type
['R', 4, ';']
['I', 2, 'b']
var
['R', 1, ',']
['I', 3, 'c']
['R', 3, ':']
['W', 7, 'array']
var_type
['R', 7, '[']
['C', 0, '1']
interval
['R', 2, '..']
['I', 4, 'a10']
Fatal: expected constant 
Fatal: invalid interval 
Fatal: invalid variable type declaration 


'invalid var declaration'

In [183]:
def scan():
    global tokens_chain, token_index, nxtsymb
    token_index += 1
    if token_index < len(tokens_chain):
        nxtsymb = tokens_chain[token_index]
        print(nxtsymb)

In [184]:
def antiscan():
    global tokens_chain, token_index, nxtsymb
    token_index -= 1
    if token_index >= 0:
        nxtsymb = tokens_chain[token_index]
        print(nxtsymb)

In [185]:
def error(text):
    print(Fore.RED + 'Fatal:', Fore.BLACK + text, Style.RESET_ALL)
#     ahah/0

In [186]:
def check(response):
    if len(response):
        error(response)
        return False
    else:
        return True

In [187]:
def wrapper():
    global nxtsymb
    scan()
    if nxtsymb[2] == 'program':
        if not check(program()):
            return 'invalid program declaration'
    scan()
    if nxtsymb[2] == 'var':
        scan()
        if not check(var()):
            return 'invalid var declaration'
        scan()
        while nxtsymb[0] == 'I':
            if not check(var()):
                return 'invalid var declaration'
            scan()   
    scan()
    scan()
    

In [188]:
def program():
    global nxtsymb
    print(Fore.GREEN + 'program' + Style.RESET_ALL)
    
    scan()
    if nxtsymb[0] != 'I':
        return 'expected identifier'
    scan()
    if nxtsymb[2] != ';':
        return 'expected ";"'
    return ''

In [189]:
def var():
    global nxtsymb
    print(Fore.GREEN + 'var' + Style.RESET_ALL)
    
    if nxtsymb[0] != 'I':
        return 'expexted identifier'
    scan()
    while nxtsymb[2] == ',':
        scan()
        if nxtsymb[0] != 'I':
            return 'expected identifier'
        scan()
    if nxtsymb[2] != ':':
        return 'expected ":"'
    scan()
    if not check(var_type()):
        return 'invalid variable type declaration'
    scan()
    if nxtsymb[2] != ';':
        return 'expected ";"'
    return ''

In [190]:
def var_type():
    global nxtsymb
    print(Fore.GREEN + 'var_type' + Style.RESET_ALL)
    
    if nxtsymb[2] in ['integer', 'real', 'string']:
        return ''
    elif nxtsymb[2] == 'array':
        scan()
        if nxtsymb[2] != '[':
            return 'expected "["'
        scan()
        if not check(interval()):
            return 'invalid interval syntax'
        scan()
        while nxtsymb[2] == ',':
            scan()
            if not check(interval()):
                return 'invalid interval'
            scan()
        if nxtsymb[2] != ']':
            return 'expected "]"'
        scan()
        if nxtsymb[2] != 'of':
            return 'expected "of"'
        scan()
        if nxtsymb[2] not in ['integer', 'real', 'string']:
            return 'expected variable type'
        return ''
        else:
            return 'invalid interval'
    else:
        return 'expected variable type of array description'
        
            

SyntaxError: invalid syntax (<ipython-input-190-eb8bf7075069>, line 29)

In [191]:
def interval():
    global nxtsymb
    print(Fore.GREEN + 'interval' + Style.RESET_ALL)
    
    if nxtsymb[0] != 'C':
        return 'expected constant'
    scan()
    if nxtsymb[2] != '..':
        return 'expected ".."'
    scan()
    if nxtsymb[0] != 'C':
        return 'expected constant'
    return ''

In [192]:
def operator():
    global nxtsymb
    
    if nxtsymb[0] == 'I':
        check(assignment())
    

In [193]:
def assignment():
    global nxtsymb
    
    if nxtsymb[0] != 'I':
        return 'expected identifier'
    scan()
    if nxtsymb[2] != ':='
        return 'expected ":="'
    scan()
    if not check(expression):
        return 'invalid expression'
    return ''

SyntaxError: invalid syntax (<ipython-input-193-91ee71dfaff2>, line 7)

In [194]:
def expression():
    global nxtsymb
    
    term()
    scan()
    while nxtsymb[2] == '+':
        scan()
        factor()
    return ''

In [195]:
def term():
    global nxtsymb
    
    factor()
    while nxtsymb[2] == '*':
        scan()
        factor()
    return ''

In [196]:
def factor():
    global nxtsymb
    
    if nxtsymb[2] == '(':
        scan()
        expression()
        if nxtsymb[2] != ')':
            return 'expected ")"'
        else:
            scan()
    else:
        if nxtsymb[0] not in ['I', 'C']:
            return 'expected constant or identifier'
        scan()
    return ''